In [1]:
# Célula 1 — importações básicas
import os
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display


In [2]:
# Célula 2B — upload manual (arquivos temporários)
from google.colab import files
uploaded = files.upload()  # Vai abrir janela para selecionar arquivos do seu PC
# Os arquivos aparecerão disponíveis em /content/
print("Arquivos enviados:", list(uploaded.keys()))


Saving clima_raw.csv to clima_raw.csv
Saving mobilidade_raw.csv to mobilidade_raw.csv
Saving trafego_raw.csv to trafego_raw.csv
Arquivos enviados: ['clima_raw.csv', 'mobilidade_raw.csv', 'trafego_raw.csv']


In [3]:
# Célula 4 — carregar CSVs
mobilidade_fp = os.path.join(RAW_DIR, 'mobilidade_raw.csv')
clima_fp = os.path.join(RAW_DIR, 'clima_raw.csv')
trafego_fp = os.path.join(RAW_DIR, 'trafego_raw.csv')

print("Caminhos usados:")
print(mobilidade_fp)
print(clima_fp)
print(trafego_fp)

# Leitura segura com try/except para identificar problemas
mobilidade = pd.read_csv(mobilidade_fp)
clima = pd.read_csv(clima_fp)
trafego = pd.read_csv(trafego_fp)

print("Mobilidade:", mobilidade.shape)
print("Clima:", clima.shape)
print("Tráfego:", trafego.shape)

display(mobilidade.head())
display(clima.head())
display(trafego.head())


NameError: name 'RAW_DIR' is not defined

In [4]:
RAW_DIR = '/content'
PROCESSED_DIR = '/content/processed'

import os
os.makedirs(PROCESSED_DIR, exist_ok=True)

print("RAW_DIR =", RAW_DIR)
print("PROCESSED_DIR =", PROCESSED_DIR)


RAW_DIR = /content
PROCESSED_DIR = /content/processed


In [5]:
# Célula 4 — carregar CSVs
mobilidade_fp = os.path.join(RAW_DIR, 'mobilidade_raw.csv')
clima_fp = os.path.join(RAW_DIR, 'clima_raw.csv')
trafego_fp = os.path.join(RAW_DIR, 'trafego_raw.csv')

print("Caminhos usados:")
print(mobilidade_fp)
print(clima_fp)
print(trafego_fp)

# Leitura segura com try/except para identificar problemas
mobilidade = pd.read_csv(mobilidade_fp)
clima = pd.read_csv(clima_fp)
trafego = pd.read_csv(trafego_fp)

print("Mobilidade:", mobilidade.shape)
print("Clima:", clima.shape)
print("Tráfego:", trafego.shape)

display(mobilidade.head())
display(clima.head())
display(trafego.head())


Caminhos usados:
/content/mobilidade_raw.csv
/content/clima_raw.csv
/content/trafego_raw.csv
Mobilidade: (3, 8)
Clima: (3, 6)
Tráfego: (3, 5)


,trip_id,route_id,timestamp,lat,lon,speed,delay,region
0,489213,8091-10,2025-05-01 07:03,-23.5102,-46.6121,12.3,3.4,Centro
1,489214,8091-10,2025-05-01 07:04,-23.5104,-46.6119,10.1,5.1,Centro
2,389111,7156-10,2025-06-21 18:12,-23.5331,-46.6442,8.2,7.0,Oeste


,datetime,rain_mm,temperature,wind_speed,humidity,region
0,2025-05-01 07:00,2.6,19.4,3.2,82,Centro
1,2025-05-01 08:00,0.0,21.1,2.5,78,Centro
2,2025-06-21 18:00,3.1,17.3,4.0,88,Oeste


,region,hour,vehicle_volume,avg_speed,jam_factor
0,Centro,7,1580,14.2,8.1
1,Centro,8,1720,12.8,9.0
2,Oeste,18,2100,11.3,9.4


In [6]:
# Célula 5 — parse de datas e criação de hour
mobilidade['timestamp'] = pd.to_datetime(mobilidade['timestamp'])
clima['datetime'] = pd.to_datetime(clima['datetime'])

mobilidade['hour'] = mobilidade['timestamp'].dt.hour
clima['hour'] = clima['datetime'].dt.hour

print("Tipos de dados:")
display(mobilidade.dtypes)
display(clima.dtypes)


Tipos de dados:


,0
trip_id,int64
route_id,object
timestamp,datetime64[ns]
lat,float64
lon,float64
speed,float64
delay,float64
region,object
hour,int32


,0
datetime,datetime64[ns]
rain_mm,float64
temperature,float64
wind_speed,float64
humidity,int64
region,object
hour,int32


In [7]:
# Célula 6 — missing values (antes)
print("Missing - mobilidade")
display(mobilidade.isna().sum())
print("Missing - clima")
display(clima.isna().sum())
print("Missing - trafego")
display(trafego.isna().sum())


Missing - mobilidade


,0
trip_id,0
route_id,0
timestamp,0
lat,0
lon,0
speed,0
delay,0
region,0
hour,0


Missing - clima


,0
datetime,0
rain_mm,0
temperature,0
wind_speed,0
humidity,0
region,0
hour,0


Missing - trafego


,0
region,0
hour,0
vehicle_volume,0
avg_speed,0
jam_factor,0


In [8]:
# Célula 7 — tratar nulos básicos e GPS inválido
# remover registros sem posição ou speed
mobilidade = mobilidade.dropna(subset=['lat','lon','speed'])
# preencher clima com forward fill por região e hora quando possível
clima = clima.sort_values(['region','datetime']).groupby('region').ffill().bfill()

# verificação pós-tratamento
print("After treatment - missing:")
display(mobilidade.isna().sum())
display(clima.isna().sum())


After treatment - missing:


,0
trip_id,0
route_id,0
timestamp,0
lat,0
lon,0
speed,0
delay,0
region,0
hour,0


,0
datetime,0
rain_mm,0
temperature,0
wind_speed,0
humidity,0
hour,0


In [9]:
# Célula 8 — outliers de speed
Q1 = mobilidade['speed'].quantile(0.25)
Q3 = mobilidade['speed'].quantile(0.75)
IQR = Q3 - Q1
low, high = Q1 - 1.5*IQR, Q3 + 1.5*IQR

print(f"Speed IQR: Q1={Q1:.2f}, Q3={Q3:.2f}, low={low:.2f}, high={high:.2f}")
before = mobilidade.shape[0]
mobilidade = mobilidade[(mobilidade['speed'] >= low) & (mobilidade['speed'] <= high)]
after = mobilidade.shape[0]
print(f"Removidos {before-after} registros como outliers de speed.")


Speed IQR: Q1=9.15, Q3=11.20, low=6.07, high=14.28
Removidos 0 registros como outliers de speed.


In [10]:
# Célula 9 — features
mobilidade['day_of_week'] = mobilidade['timestamp'].dt.dayofweek  # 0=segunda
mobilidade['is_peak'] = mobilidade['hour'].isin([7,8,9,17,18,19]).astype(int)

# criar rain_category (se existir rain_mm após merge; colocamos categ aqui só se for necessário depois)
# Exemplo de normalização do delay (se existir)
if 'delay' in mobilidade.columns:
    mobilidade['delay_clip'] = mobilidade['delay'].clip(lower=-30, upper=120)  # limites razoáveis


In [11]:
print("Colunas mobilidade:", mobilidade.columns.tolist())
print("Colunas clima:", clima.columns.tolist())
print("Colunas trafego:", trafego.columns.tolist())

# Se 'region' não existir em mobilidade, talvez precise mapear via lat/lon para shapefile;
# aqui assumimos que region existe. Caso contrário, peça ajuda para geocodificação.


Colunas mobilidade: ['trip_id', 'route_id', 'timestamp', 'lat', 'lon', 'speed', 'delay', 'region', 'hour', 'day_of_week', 'is_peak', 'delay_clip']
Colunas clima: ['datetime', 'rain_mm', 'temperature', 'wind_speed', 'humidity', 'hour']
Colunas trafego: ['region', 'hour', 'vehicle_volume', 'avg_speed', 'jam_factor']


In [12]:
# Célula 11 — merge
df = (
    mobilidade
    .merge(
        clima[['region','hour','rain_mm','temperature','wind_speed']],
        on=['region','hour'],
        how='left'
    )
    .merge(
        trafego, on=['region','hour'], how='left', suffixes=('', '_trafego')
    )
)

print("Dataset final shape:", df.shape)
display(df.head())


KeyError: "['region'] not in index"

In [13]:
clima.head()
clima.columns


Index(['datetime', 'rain_mm', 'temperature', 'wind_speed', 'humidity', 'hour'], dtype='object')

In [14]:
mobilidade['region'].unique()


array(['Centro', 'Oeste'], dtype=object)

In [15]:
clima['region'] = 'Centro'


In [16]:
df = (
    mobilidade
    .merge(clima[['region','hour','rain_mm','temperature','wind_speed']],
           on=['region','hour'], how='left')
    .merge(trafego, on=['region','hour'], how='left')
)


In [17]:
# Célula 12 — checagens finais
print("Resumo estatístico:")
display(df.describe(include='all'))
print("Missing values após merge:")
display(df.isna().sum().sort_values(ascending=False).head(20))


Resumo estatístico:


,trip_id,route_id,timestamp,lat,lon,speed,delay,region,hour,day_of_week,is_peak,delay_clip,rain_mm,temperature,wind_speed,vehicle_volume,avg_speed,jam_factor
count,3.000000,3,3,3.000000,3.000000,3.000000,3.000000,3,3.000000,3.000000,3.0,3.000000,2.0,2.0,2.0,3.000000,3.000000,3.000000
unique,NaN,2,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,8091-10,NaN,NaN,NaN,NaN,NaN,Centro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,455846.000000,NaN,2025-05-18 10:46:20,-23.517900,-46.622733,10.200000,5.166667,NaN,10.666667,3.666667,1.0,5.166667,2.6,19.4,3.2,1753.333333,13.233333,8.533333
min,389111.000000,NaN,2025-05-01 07:03:00,-23.533100,-46.644200,8.200000,3.400000,NaN,7.000000,3.000000,1.0,3.400000,2.6,19.4,3.2,1580.000000,11.300000,8.100000
25%,439162.000000,NaN,2025-05-01 07:03:30,-23.521750,-46.628150,9.150000,4.250000,NaN,7.000000,3.000000,1.0,4.250000,2.6,19.4,3.2,1580.000000,12.750000,8.100000
50%,489213.000000,NaN,2025-05-01 07:04:00,-23.510400,-46.612100,10.100000,5.100000,NaN,7.000000,3.000000,1.0,5.100000,2.6,19.4,3.2,1580.000000,14.200000,8.100000
75%,489213.500000,NaN,2025-05-27 00:38:00,-23.510300,-46.612000,11.200000,6.050000,NaN,12.500000,4.000000,1.0,6.050000,2.6,19.4,3.2,1840.000000,14.200000,8.750000
max,489214.000000,NaN,2025-06-21 18:12:00,-23.510200,-46.611900,12.300000,7.000000,NaN,18.000000,5.000000,1.0,7.000000,2.6,19.4,3.2,2100.000000,14.200000,9.400000


Missing values após merge:


,0
wind_speed,1
rain_mm,1
temperature,1
trip_id,0
route_id,0
timestamp,0
delay,0
lat,0
lon,0
speed,0


In [18]:
# Célula 13 — salvar arquivo processado
processed_fp = os.path.join(PROCESSED_DIR, 'dataset_processado.csv')
df.to_csv(processed_fp, index=False)
print("Arquivo salvo em:", processed_fp)


Arquivo salvo em: /content/processed/dataset_processado.csv


In [19]:
# Célula 14 — download local (apenas se não usou Drive)
from google.colab import files
files.download(processed_fp)  # abrirá o diálogo de download


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>